In [1]:
from dotenv import load_dotenv
from functions.MySQL_io import MySQLConnection
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
load_dotenv()

user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')
database = os.getenv('DATABASE')
port = os.getenv('SQL_PORT')
server = os.getenv('SQL_SERVER')

In [5]:
con = MySQLConnection(port, database, user, password, server)

lap_data = con.run_query(query='''
    SELECT
        IFNULL(LapTimeFP1, 0) AS LapTimeFP1,
        IFNULL(CompoundFP1, 0) AS CompoundFP1,
        IFNULL(AirTempFP1, 0) AS AirTempFP1,
        IFNULL(RainfallFP1, 0) AS RainfallFP1,
        IFNULL(TrackTempFP1, 0) AS TrackTempFP1,
        CASE WHEN isnull(LapTimeFP1) THEN 1 ELSE 0 END AS FP1_Missing_Flag,
        IFNULL(LapTimeFP2, 0) AS LapTimeFP2,
        IFNULL(CompoundFP2, 0) AS CompoundFP2,
        IFNULL(AirTempFP2, 0) AS AirTempFP2,
        IFNULL(RainfallFP2, 0) AS RainfallFP2,
        IFNULL(TrackTempFP2, 0) AS TrackTempFP2,
        CASE WHEN year <= 2022 THEN CASE WHEN isnull(LapTimeFP2) THEN 1 ELSE 0 END ELSE 0 END FP2_Missing_Flag,
        IFNULL(LapTimeFP3, 0) AS LapTimeFP3,
        IFNULL(CompoundFP3, 0) AS CompoundFP3,
        IFNULL(AirTempFP3, 0) AS AirTempFP3,
        IFNULL(RainfallFP3, 0) AS RainfallFP3,
        IFNULL(TrackTempFP3, 0) AS TrackTempFP3,
        0 AS FP3_Missing_Flag,
        LapTimeQ,
        AirTempQ,
        RainfallQ,
        sprint_flag as is_sprint,
        traction,
        tyre_stress,
        asphalt_grip,
        braking,
        asphalt_abrasion,
        lateral_force,
        track_evolution,
        downforce
    FROM cleaned_session_data
    where sprint_flag = 1 and
    LapTimeQ is not NULL and
    event_cd = 18 and year = 2024
    UNION ALL
    SELECT
        IFNULL(LapTimeFP1, 0) AS LapTimeFP1,
        IFNULL(CompoundFP1, 0) AS CompoundFP1,
        IFNULL(AirTempFP1, 0) AS AirTempFP1,
        IFNULL(RainfallFP1, 0) AS RainfallFP1,
        IFNULL(TrackTempFP1, 0) AS TrackTempFP1,
        CASE WHEN isnull(LapTimeFP1) THEN 1 ELSE 0 END AS FP1_Missing_Flag,
        IFNULL(LapTimeFP2, 0) AS LapTimeFP2,
        IFNULL(CompoundFP2, 0) AS CompoundFP2,
        IFNULL(AirTempFP2, 0) AS AirTempFP2,
        IFNULL(RainfallFP2, 0) AS RainfallFP2,
        IFNULL(TrackTempFP2, 0) AS TrackTempFP2,
        CASE WHEN isnull(LapTimeFP2) THEN 1 ELSE 0 END AS FP2_Missing_Flag,
        IFNULL(LapTimeFP3, 0) AS LapTimeFP3,
        IFNULL(CompoundFP3, 0) AS CompoundFP3,
        IFNULL(AirTempFP3, 0) AS AirTempFP3,
        IFNULL(RainfallFP3, 0) AS RainfallFP3,
        IFNULL(TrackTempFP3, 0) AS TrackTempFP3,
        CASE WHEN isnull(LapTimeFP3) THEN 1 ELSE 0 END AS FP3_Missing_Flag,
        LapTimeQ,
        AirTempQ,
        RainfallQ,
        sprint_flag as is_sprint,
        traction,
        tyre_stress,
        asphalt_grip,
        braking,
        asphalt_abrasion,
        lateral_force,
        track_evolution,
        downforce
    FROM cleaned_session_data
    where sprint_flag = 0 and
    LapTimeQ is not NULL and
    concat(year, event_cd) != 202418
    ''')

con.close_connection()

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'LapTimeFP1' in 'field list'
[SQL: 
    SELECT
        IFNULL(LapTimeFP1, 0) AS LapTimeFP1,
        IFNULL(CompoundFP1, 0) AS CompoundFP1,
        IFNULL(AirTempFP1, 0) AS AirTempFP1,
        IFNULL(RainfallFP1, 0) AS RainfallFP1,
        IFNULL(TrackTempFP1, 0) AS TrackTempFP1,
        CASE WHEN isnull(LapTimeFP1) THEN 1 ELSE 0 END AS FP1_Missing_Flag,
        IFNULL(LapTimeFP2, 0) AS LapTimeFP2,
        IFNULL(CompoundFP2, 0) AS CompoundFP2,
        IFNULL(AirTempFP2, 0) AS AirTempFP2,
        IFNULL(RainfallFP2, 0) AS RainfallFP2,
        IFNULL(TrackTempFP2, 0) AS TrackTempFP2,
        CASE WHEN year <= 2022 THEN CASE WHEN isnull(LapTimeFP2) THEN 1 ELSE 0 END ELSE 0 END FP2_Missing_Flag,
        IFNULL(LapTimeFP3, 0) AS LapTimeFP3,
        IFNULL(CompoundFP3, 0) AS CompoundFP3,
        IFNULL(AirTempFP3, 0) AS AirTempFP3,
        IFNULL(RainfallFP3, 0) AS RainfallFP3,
        IFNULL(TrackTempFP

# Data Cleaning

In [ ]:
lap_data = lap_data.astype(float)

In [ ]:
lap_data.count()

In [ ]:
lap_data.head()

In [ ]:
lap_data.isna().sum()

### Test Train Split (80 - 20)

In [ ]:
train_data = lap_data.sample(frac=0.8, random_state=0)
test_data = lap_data.drop(train_data.index)

# Data Exploration

In [ ]:
sns.pairplot(train_data[['LapTimeFP1', 'LapTimeFP2', 'LapTimeFP3', 'LapTimeQ']], diag_kind='kde')

In [ ]:
train_data.describe().transpose()

# Model Prep

In [ ]:
train_features = train_data.copy()
test_features = test_data.copy()

train_labels = train_features.pop('LapTimeQ')
test_labels = test_features.pop('LapTimeQ')

### Normalization

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
normalizer.adapt(np.array(train_features))

In [ ]:
normalizer.mean.numpy()

# DNN Model

In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(32, activation='relu'),
      layers.Dense(16, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

type(dnn_model)

In [ ]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.1,
    verbose=0, epochs=1000)

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, 10])
plt.xlabel('Epoch')
plt.ylabel('Error [LapTimeQ]')
plt.legend()
plt.grid(True)

In [ ]:
dnn_model.summary()

### Save Results

In [ ]:
test_results = pd.DataFrame()

test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

In [ ]:
test_results.to_csv('dnn_model.csv')

In [ ]:
dnn_model.save('dnn_model.keras')

### Evaluation

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [LapTimeQ]')
plt.ylabel('Predictions [LapTimeQ]')
lims = [0, 150]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [LapTimeQ]')
_ = plt.ylabel('Count')

In [ ]:
lap_data_test = con.run_query(query='''
    SELECT
        DriverNumber,
        IFNULL(LapTimeFP1, 0) AS LapTimeFP1,
        IFNULL(CompoundFP1, 0) AS CompoundFP1,
        IFNULL(AirTempFP1, 0) AS AirTempFP1,
        IFNULL(RainfallFP1, 0) AS RainfallFP1,
        IFNULL(TrackTempFP1, 0) AS TrackTempFP1,
        CASE WHEN isnull(LapTimeFP1) THEN 1 ELSE 0 END AS FP1_Missing_Flag,
        IFNULL(LapTimeFP2, 0) AS LapTimeFP2,
        IFNULL(CompoundFP2, 0) AS CompoundFP2,
        IFNULL(AirTempFP2, 0) AS AirTempFP2,
        IFNULL(RainfallFP2, 0) AS RainfallFP2,
        IFNULL(TrackTempFP2, 0) AS TrackTempFP2,
        CASE WHEN isnull(LapTimeFP2) THEN 1 ELSE 0 END AS FP2_Missing_Flag,
        IFNULL(LapTimeFP3, 0) AS LapTimeFP3,
        IFNULL(CompoundFP3, 0) AS CompoundFP3,
        IFNULL(AirTempFP3, 0) AS AirTempFP3,
        IFNULL(RainfallFP3, 0) AS RainfallFP3,
        IFNULL(TrackTempFP3, 0) AS TrackTempFP3,
        CASE WHEN isnull(LapTimeFP3) THEN 1 ELSE 0 END AS FP3_Missing_Flag,
        LapTimeQ,
        AirTempQ,
        RainfallQ,
        sprint_flag as is_sprint,
        traction,
        tyre_stress,
        asphalt_grip,
        braking,
        asphalt_abrasion,
        lateral_force,
        track_evolution,
        downforce
    FROM ml_project_prod.cleaned_session_data
    WHERE
	event_cd = 18 and
    year = 2024
    ''')

con.close_connection()

In [ ]:
test_features_actual = lap_data_test.copy()
test_labels_actual = test_features_actual[['LapTimeQ', 'DriverNumber']]
test_features_actual.pop('LapTimeQ')

In [ ]:
predict_df = pd.DataFrame()
for i in range(len(test_features_actual)):
    vals = pd.DataFrame()
    temp = test_features_actual.iloc[i].to_frame().transpose()
    temp = temp.astype(float)
    predicted_time = dnn_model.predict(temp.drop(['DriverNumber'], axis = 1)).flatten()
    vals['DriverNumber'] = temp['DriverNumber']
    vals['predicted_time'] = predicted_time
    predict_df = pd.concat([predict_df, vals])

In [ ]:
predict_df = predict_df.sort_values('predicted_time')
predict_df['Predicted_POS'] = predict_df['predicted_time'].rank(method='first')

In [ ]:
test_labels_actual = test_labels_actual.sort_values('LapTimeQ')
test_labels_actual['actual_POS'] = test_labels_actual['LapTimeQ'].rank(method='first')

In [ ]:
final_df = pd.merge(test_labels_actual, predict_df, on='DriverNumber')

In [ ]:
final_df['Laptime_dif'] = final_df['LapTimeQ'] - final_df['predicted_time']

In [ ]:
final_df['POS_dif'] = final_df['actual_POS'] - final_df['Predicted_POS']

In [ ]:
final_df

In [ ]:
a = plt.axes(aspect='equal')asd
plt.scatter(final_df['LapTimeQ'], final_df['predicted_time'])
plt.xlabel('True Values [LapTimeQ]')
plt.ylabel('Predictions [predicted_time]')
lims = [0, 150]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
a = plt.axes(aspect='equal')
plt.scatter(final_df['actual_POS'], final_df['Predicted_POS'])
plt.xlabel('True Values [POS]')
plt.ylabel('Predictions [POS]')
lims = [0, 20]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
error = final_df['actual_POS'] - final_df['Predicted_POS']
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [LapTimeQ]')
_ = plt.ylabel('Count')

In [ ]:
error = final_df['LapTimeQ'] - final_df['predicted_time']
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [LapTimeQ]')
_ = plt.ylabel('Count')